In [1]:
# Source: https://github.com/prestinomills/aqueduct/blob/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/Active_Business_Finalized_Script.py
# %load active_business_script.py
"""
Created on Wed May  1 08:51:03 2019
@author: myrfid041
"""
import geopandas as gpd
import os
import pandas as pd

from sodapy import Socrata
from arcgis.gis import GIS
from arcgis.features.summarize_data import join_features
from IPython.display import display
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
from copy import deepcopy

lahub_user = os.environ["LAHUB_ACC_USERNAME"]
lahub_pass = os.environ["LAHUB_ACC_PASSWORD"]


#---Setting the Outputs
OUTPUT_FILE = "./Listing_of_Active_Businesses.csv"


#---Pulling Active Business Data
#client = Socrata("data.lacity.org", None)
#abiz = pd.DataFrame(client.get('ngkp-kqkn', limit=10000000)).to_pickle("../data/abiz.p")
abiz = pd.read_pickle("../data/abiz.p")
#---Pull NAIC Industry Table
#n_table='https://raw.githubusercontent.com/prestinomills/aqueduct/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/naics_industry_table.csv'
naics_table=pd.read_csv("../data/naics_industry_table.csv")


#BLOCK_GROUPS_2020 = "https://opendata.arcgis.com/datasets/51f14a2885794cf2a487b5057f149086_15.geojson"
#gpd.read_file(BLOCK_GROUPS_2020).head()

In [2]:
def chunks(l, n, z):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        chunk_list=l[i:i + n]
        z.edit_features(updates= chunk_list)
        print("update successful")


def top10(x):
    x.pred_ind=x.idxmax(axis=1)
    predom_industries=pd.DataFrame(x.pred_ind.value_counts())[0:12]
    predom_industries=predom_industries.index.tolist()

    drop_me = ['Professional, Scientific, and Technical Services', 
        'Other Services (except Public Administration)'
    ]

    #for i in drop_me:
    #    predom_industries.remove(i)

    return predom_industries


def dataprep(df,naics_table):
    # Grab location info
    df = (df.dropna(subset=['location_1', 'naics'])
        .assign(
            location_2 = df.location_1.astype(str).str[34:-2]
        )
    )

    df = df.assign(
        longitude = df.location_2.str.split(",", expand=True)[0].astype(float),
        latitude = df.location_2.str.split(",", expand=True)[1].astype(float),
        naics_sector = df.naics.str[:2].astype(str),
    ).dropna(subset=["longitude", "latitude"])


    # Merge in NAICS sector
    df2 = pd.merge(df, 
                   naics_table.assign(
                       naics_sector = naics_table.naics_sector.astype(str)
                   ), 
            how = 'inner', on = 'naics_sector', validate = 'm:1'
            )

    # Create geometry column
    gdf = gpd.GeoDataFrame(df2.dropna(subset=['longitude', 'latitude']), 
        geometry = gpd.points_from_xy(df2.longitude, df2.latitude),
                                      crs = "EPSG:4326"
    ).to_crs("EPSG:2229") # Change to CA State Plane

    # Block groups
    # This URL needs to change, need to upload files to right repo
    # https://github.com/prestinomills/aqueduct/tree/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/LACounty_Blockgroup
    block = gpd.read_file("../data/LACounty_Blockgroup.geojson").to_crs("EPSG:2229")

    # Aggregate
    sjoin=gpd.sjoin(gdf, block, how='inner', op='intersects')
    
    sjoin = sjoin.assign(
        GEOID10 = sjoin.GEOID10.astype(str).apply(lambda x: '{0:0>12}'.format(x))
    )
    
    sjoin2=sjoin.pivot_table(index='GEOID10', values='business_name',
        columns=['naics_industry'], aggfunc=len)

    #sjoin2 = sjoin2.fillna(0).assign(
    #    GEOID10 = sjoin2.GEOID10.astype(str).apply(lambda x: '{0:0>12}'.format(x))
    #)
    #sjoin2.to_csv(OUTPUT_FILE)
    #predom_ind=top10(sjoin2)

    #return sjoin2, predom_ind
    return sjoin


In [3]:
t1 = dataprep(abiz, naics_table)

In [ ]:
# Figure out what the idx is
# What is the purpose of that portion...if it's to find which one is predominant industry
# Not quite sure what the top 10 predominant industry function is returning
# is it returning values to replace in the output of the top 10 predominant?
# what happens if one industry was in top 10 in previous period, but no longer? what kind of value is replaced?

In [4]:
t2=(t1.pivot_table(index='GEOID10', values='business_name',
    columns=['naics_industry'], aggfunc=len).reset_index()
    .fillna(0)
   )

In [5]:
predominant = t2.set_index("GEOID10").idxmax(axis=1)

In [6]:
# Is this the desired output? 
predominant

GEOID10
060371011101                                         Construction
060371011102                                         Retail Trade
060371011103                                         Construction
060371011221     Professional, Scientific, and Technical Services
060371011222     Professional, Scientific, and Technical Services
                                      ...                        
060379800261    Administrative and Support and Waste Managemen...
060379800281                      Accommodation and Food Services
060379800301                  Arts, Entertainment, and Recreation
060379800311                                         Retail Trade
060379800331                                         Construction
Length: 6297, dtype: object

In [7]:
pd.DataFrame(predominant).reset_index().rename(columns = {0: "predominant_industry"})

,GEOID10,predominant_industry
0,060371011101,Construction
1,060371011102,Retail Trade
2,060371011103,Construction
3,060371011221,"Professional, Scientific, and Technical Services"
4,060371011222,"Professional, Scientific, and Technical Services"
...,...,...
6292,060379800261,Administrative and Support and Waste Managemen...
6293,060379800281,Accommodation and Food Services
6294,060379800301,"Arts, Entertainment, and Recreation"
6295,060379800311,Retail Trade


In [ ]:
def top10(x):
    x.pred_ind=x.idxmax(axis=1)
    predom_industries=pd.DataFrame(x.pred_ind.value_counts())[0:12]
    predom_industries=predom_industries.index.tolist()

    drop_me = ['Professional, Scientific, and Technical Services', 
        'Other Services (except Public Administration)'
    ]

    #for i in drop_me:
    #    predom_industries.remove(i)

    return predom_industries

In [ ]:
#predom_ind=top10(t3)

In [ ]:
def go():
    df, predom=dataprep(abiz,naics_table)
    geohub_updates(df,lahub_user,lahub_pass,predom)


def update_desc(x,y,z):
    active_biz=z.content.get(y)    
    text = """
    This layer is aggregating <a href="https://data.lacity.org/A-Prosperous-City/Listing-of-Active-Businesses/6rrh-rzua">Listing of Active Businesses Data</a> that have geospatial information associated. The top 10 most frequent industries in block groups are:
    {}
    """
    
    item_props = {'title' : 'Active Businesses Data by Block Group', 'description':text.format(x)}
    active_biz.update(item_properties=item_props)
    print("updates made!")


def geohub_updates(x,user,pas,topz):
    gis = GIS('https://lahub.maps.arcgis.com',  username=user, password=pas)
    output_layer_name = '067a9242fbef4afeb1ca0744952e5724'
    actbus=gis.content.search(output_layer_name)
    ActiveBusinesses_item = actbus[0]
    ActiveBusinesses_flayer = ActiveBusinesses_item.layers[0]
    ActiveBusinesses_fset = ActiveBusinesses_flayer.query() #querying without any conditions returns all the features
    overlap_rows = pd.merge(ActiveBusinesses_fset.sdf, 
                        x, 
                        how='inner',
                        on = 'GEOID10'
                        )


    #Perform updates

    features_for_update = [] #list containing corrected features
    all_features = ActiveBusinesses_fset.features

    for GEOID10 in overlap_rows['GEOID10']:
        # get the feature to be updated
        original_feature = [f for f in all_features if f.attributes['GEOID10'] == GEOID10][0]
        feature_to_be_updated = deepcopy(original_feature)

        # get the matching row from csv
        matching_row = x.where(x.GEOID10 == GEOID10).dropna()

        # assign the updated values
        layer_attribute_dict = {
            "Accommodation_and_Food_Services": 'Accommodation and Food Services',
            'Administrative_and_Support_and_': 'Administrative and Support and Waste Management and Remediation Services'
            'Agriculture__Forestry__Fishing_': 'Agriculture, Forestry, Fishing and Hunting',
            'Arts__Entertainment__and_Recrea': 'Arts, Entertainment, and Recreation', 
            'Construction': 'Construction', 
            'Educational_Services': 'Educational Services', 
            'Finance_and_Insurance': 'Finance and Insurance',
            'Health_Care_and_Social_Assistan': 'Health Care and Social Assistance', 
            'Information': 'Information', 
            'Manufacturing': 'Manufacturing', 
            'Medical_Marijuana_Collective': 'Medical Marijuana Collective', 
            'Mining': 'Mining', 
            'Not_Classified': 'Not Classified', 
            'Other_Services__except_Public_A': 'Other Services (except Public Administration)', 
            'Professional__Scientific__and_T': 'Professional, Scientific, and Technical Services', 
            'Real_Estate_Rental_and_Leasing': 'Real Estate Rental and Leasing', 
            'Retail_Trade': 'Retail Trade', 
            'Transportation_and_Warehousing': 'Transportation and Warehousing', 
            'Utilities': 'Utilities', 
            'Wholesale_Trade': 'Wholesale Trade',                                                 
        }

        for key, value in layer_attribute_dict.items():
            feature_to_be_updated.attributes[key] = matching_row[value].values[0]

        #add this to the list of features to be updated
        features_for_update.append(feature_to_be_updated)
    chunks(features_for_update, 1000,ActiveBusinesses_flayer)
    update_desc(topz,output_layer_name,gis)